## MainQuest 01, MyGPT

### 1. 데이터 전처리

- 질문 데이터와 답변 데이터를 모두 학습용 데이터(디코더의 입력 데이터)로 사용
- 문장부호, 숫자, 알파벳 단어, 한글 단어 단위로 토큰화
- SubwordTextEncoder 기반 토크나이저 대신, 공백을 기준으로 토큰을 생성 (한글 단어잘림 문제 방지)

### 2. 모델 구성

###### 트랜스포머와 다른 점
- 인코더 레이어(encoder_layer)와 인코더(encoder) 완전 삭제
- 디코더 레이어(decoder_layer)와 디코더(decoder) 새롭게 구성
- GPT의 디코더 레이어는 MHA(Multi-head Attention) - LN(LayerNormalization) - Dense - LN 구성임
- 트랜스포머의 디코더 레이어는 MHA - LN - MHA - Dropout - LN 구성
- 트랜스포머의 디코더 레이어에 있는 Multi-head Attention 계층 통과후 Dropout 레이어, Dense 레이어 구성이 GPT에서는 삭제 (논문에 설명 없음)

###### 논문 구성과 다른 점
- GPT의 디코더의 출력은 Text Prediction을 위한 Dense 레이어와 Task Classification을 위한 Dense 레이어로 구성
- 구현된 모델에서는 Task Classification을 위한 Dense 레이어는 없음 (pre-train part만 구현)
- 하이퍼파라미터 구성은 논문과 다름 (입력 차원 수, 어텐션 헤드 수, 러닝 레이트 스케줄러 등)

### 3. 학습 및 테스트, 하이퍼파라미터

- 빠른 학습을 위해 데이터셋을 1000개만 가지고 옴
- 하이퍼파라미터 구성 중 UNIT(내부 dense 레이어의 unit 수), DROPOUT (드랍아웃 비율)은 사용되지 않음 (관련 레이어 삭제됨)

### 1. 데이터 전처리

In [203]:
import tensorflow as tf
import pandas as pd
import re

In [204]:
# 학습 데이터 읽어오기
data = pd.read_csv("ChatbotData.csv")[:1000]
data.head()
# label: 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [205]:
# 문장 단위 텍스트 전처리
def preprocess_sentence(sentence):
    # 소문자 변환
    sentence = sentence.lower()

    # 문장 부호 간 띄어쓰기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    # 숫자 간 띄어쓰기
    sentence = re.sub(r"([0-9]+)", r" \1 ", sentence)
    
    # 알파벳 한글 간 띄어쓰기
    sentence = re.sub(r"([a-zA-Z]+)", r" \1 ", sentence)

    # 알파벳, 한글, 숫자, 문장부호를 제외한 모든 문자를 공백으로 대체
    sentence = re.sub("[^a-zA-Z0-9가-힣.?!,]", " ", sentence)

    # 빈 칸 반복되는 것 하나로 줄이기
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 문장 앞 뒤 공백 자르기
    sentence = sentence.strip()

    return sentence

In [206]:
# 데이터셋 단위 전처리
def preprocess_data(data):
    res = []
    for q, a in zip(data['Q'], data['A']):
        res.append(preprocess_sentence(q))
        res.append(preprocess_sentence(a))

    return res

In [207]:
# 데이터셋 구성
sentences = preprocess_data(data)
print(len(sentences))

2000


In [208]:
# 단어 사전 생성 -> 띄어쓰기 단위로 토큰을 분리하고 카운팅
UNKNOWN_TOKEN = '<unknown>'
START_TOKEN = '<start>'
END_TOKEN = '<end>'
def space_tokenizer(data):
    dic = {UNKNOWN_TOKEN: 1, START_TOKEN: 2, END_TOKEN: 3}
    index = 4
    for sentence in data:
        for word in sentence.split(" "):
            if not word in dic:
                dic[word] = index
                index += 1
    return dic

In [234]:
word_dic = space_tokenizer(sentences)
VOCAB_SIZE = len(word_dic) + 1
print(f'단어 사전의 크기: {VOCAB_SIZE}')

단어 사전의 크기: 2834


In [235]:
# 토큰화 - 문장 단위 (시작 토큰, 종료 토큰 추가됨)
def tokenize(sentence):
    res = []
    sentence = START_TOKEN + " " + sentence + " " + END_TOKEN
    for word in sentence.split(" "):
        res.append(word_dic[word])

    return res

In [236]:
# 토큰화 - 데이터 단위
def tokenize_data(data):
    res = []
    for sentence in data:
        res.append(tokenize(sentence))
    
    return res

In [237]:
# 토큰화
inputs = tokenize_data(sentences)

In [238]:
# 최대 길이 구하기
def max_array_length(data):
    max_len = 0
    
    for element in data:
        if len(element) > max_len:
            max_len = len(element)
            
    return max_len

In [239]:
MAX_LENGTH = max_array_length(inputs)
print(f'토큰화된 문장의 최대 길이: {MAX_LENGTH}')

토큰화된 문장의 최대 길이: 15


In [240]:
# 패딩 - 데이터가 패딩 길이보다 짧으면 삭제됨
def padding(data, pad_len):
    res = []

    for sentence in data:
        if len(sentence) <= pad_len:
            res.append(sentence)

    res = tf.keras.preprocessing.sequence.pad_sequences(res, maxlen=pad_len, padding='post')

    return res

In [241]:
inputs = padding(inputs, MAX_LENGTH)
print(f'패딩된 데이터의 차원: {inputs.shape}')

패딩된 데이터의 차원: (2000, 15)


## 2. 모델 구성

In [242]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        # 각도 배열 생성
        angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis], i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :], d_model=d_model)

        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # sin과 cosine이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])

        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [243]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

    # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)

    return output

In [244]:
# 멀티헤드 어텐션
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))

        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

In [245]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [246]:
# 디코더 레이어
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어: Masked Multi Self Attention
    attention = MultiHeadAttention(d_model, num_heads, name="attention")(inputs={'query': inputs, 'key': inputs, 'value': inputs, 'mask': look_ahead_mask})

    # 두 번째 서브 레이터: Layer Normalization
    normalized_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + inputs)

    # 세 번째 서브 레이어: Feed Forward -> Layer Normalization 결과와 합치기 위해 feed_forward 레이어의 출력이 d_model과 같아야 함
    feed_forward = tf.keras.layers.Dense(d_model, name="feed_forward")(inputs=normalized_attention)

    # 네 번째 서브 레이어: Layer Normalization
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(feed_forward + normalized_attention)

    # Dropout 필요한가?
    #attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)

    # 출력 레이어 1: 2개의 완전연결층
    #outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    #outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 출력 레이어 2: 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    #outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    #outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(inputs=[inputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

In [247]:
# 디코더
def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')

    # 마스크
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(units=units, d_model=d_model, num_heads=num_heads, dropout=dropout, name='decoder_layer_{}'.format(i))(inputs=[outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(inputs=[inputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

In [248]:
# MyGPT
def mygpt(vocab_size, num_layers, units, d_model, num_heads, dropout, name="mygpt"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape=(1, None, None), name='look_ahead_mask')(inputs)
    padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='padding_mask')(inputs)

    # 디코더
    outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, units=units, d_model=d_model, num_heads=num_heads, dropout=dropout)(inputs=[inputs, look_ahead_mask, padding_mask])

    # 완전연결층 - 논문에 의하면 독립된 완전 연결층이 2개 필요 -> pretrain 단계만 구현
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(outputs)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name=name)

In [249]:
# 하이퍼파라미터
NUM_LAYERS = 2 # 디코더의 층의 개수
D_MODEL = 256 # 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기 -> 사용하지 않음
DROPOUT = 0.1 # 드롭아웃의 비율 -> 사용하지 않음

# 모델
model = mygpt(vocab_size=VOCAB_SIZE, num_layers=NUM_LAYERS, units=UNITS, d_model=D_MODEL, num_heads=NUM_HEADS, dropout=DROPOUT)
model.summary()

Model: "mygpt"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
padding_mask (Lambda)           (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 256)    1385472     inputs[0][0]                     
                                                                 look_ahead_mask[0][0]        

### 3. 학습 및 테스트, 하이퍼파라미터

In [250]:
# Loss 함수
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [251]:
# Learning rate 스케줄러
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [252]:
# 옵티마이져
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [253]:
# Accuracy 함수
def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [254]:
# 모델 학습을 위한 데이터셋 준비
BATCH_SIZE = 64
BUFFER_SIZE = 20000

trainset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': inputs[:, :-1]
    },
    {
        'outputs': inputs[:, 1:]
    },
))

trainset = trainset.cache()
trainset = trainset.shuffle(BUFFER_SIZE)
trainset = trainset.batch(BATCH_SIZE)
trainset = trainset.prefetch(tf.data.experimental.AUTOTUNE)

In [255]:
# 학습
EPOCHS = 10

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
history = model.fit(trainset, epochs=EPOCHS, verbose=1)

Epoch 1/10
32/32 [==============================] - 11s 268ms/step - loss: 2.8423 - accuracy: 7.1429e-05
Epoch 2/10
32/32 [==============================] - 9s 273ms/step - loss: 2.7486 - accuracy: 0.0108
Epoch 3/10
32/32 [==============================] - 9s 267ms/step - loss: 2.5919 - accuracy: 0.0703
Epoch 4/10
32/32 [==============================] - 9s 268ms/step - loss: 2.4399 - accuracy: 0.0714
Epoch 5/10
32/32 [==============================] - 8s 265ms/step - loss: 2.3272 - accuracy: 0.0714
Epoch 6/10
32/32 [==============================] - 9s 268ms/step - loss: 2.2443 - accuracy: 0.0714
Epoch 7/10
32/32 [==============================] - 9s 272ms/step - loss: 2.1747 - accuracy: 0.0715
Epoch 8/10
32/32 [==============================] - 9s 273ms/step - loss: 2.1103 - accuracy: 0.0747
Epoch 9/10
32/32 [==============================] - 9s 273ms/step - loss: 2.0471 - accuracy: 0.0821
Epoch 10/10
32/32 [==============================] - 9s 269ms/step - loss: 1.9869 - accuracy: 0